In [1]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV, cross_val_score

import pandas as pd
import numpy as np
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
X_train = pd.read_csv('../data/X_train_flu.csv', index_col= 'respondent_id')
X_test = pd.read_csv('../data/X_test_flu.csv', index_col= 'respondent_id')
y_train = pd.read_csv('../data/y_train_flu.csv', index_col= 'respondent_id')
y_test = pd.read_csv('../data/y_test_flu.csv', index_col= 'respondent_id')

In [3]:
y_train = y_train['seasonal_vaccine']
y_test = y_test['seasonal_vaccine']

In [4]:
X_train.head()

,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,...,race,sex,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children
respondent_id,,,,,,,,,,,,,,,,,,,,,
7273,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,Other or Multiple,Female,"<= $75,000, Above Poverty",Married,Own,Not in Labor Force,qufhixun,"MSA, Not Principle City",1.0,0.0
13773,2.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,...,White,Female,NaN,Not Married,Own,Employed,lrircsnp,Non-MSA,0.0,0.0
24162,1.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,...,White,Female,"> $75,000",Married,Own,Employed,bhuqouqj,"MSA, Not Principle City",1.0,0.0
17341,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,...,Black,Female,"<= $75,000, Above Poverty",Not Married,Own,Employed,fpwskwrf,"MSA, Principle City",1.0,2.0
23521,2.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,...,White,Male,"<= $75,000, Above Poverty",Married,Own,Not in Labor Force,lzgpxyit,"MSA, Not Principle City",1.0,0.0


In [5]:
X_train.isna().sum()

h1n1_concern                     69
h1n1_knowledge                   85
behavioral_antiviral_meds        51
behavioral_avoidance            155
behavioral_face_mask             14
behavioral_wash_hands            30
behavioral_large_gatherings      55
behavioral_outside_home          55
behavioral_touch_face            90
doctor_recc_h1n1               1605
doctor_recc_seasonal           1605
chronic_med_condition           745
child_under_6_months            617
health_worker                   606
opinion_h1n1_vacc_effective     306
opinion_h1n1_risk               290
opinion_h1n1_sick_from_vacc     293
opinion_seas_vacc_effective     350
opinion_seas_risk               394
opinion_seas_sick_from_vacc     414
age_group                         0
education                      1048
race                              0
sex                               0
income_poverty                 3261
marital_status                 1053
rent_or_own                    1518
employment_status           

In [6]:
y_train.isna().sum()

0

In [7]:
X_train_cat = X_train.select_dtypes('object')
X_train_num = X_train.select_dtypes(['float64', 'int64'])

cat_pipe = Pipeline([
    ('impute', SimpleImputer(strategy='most_frequent')),
    ('ohe', OneHotEncoder(sparse=False, handle_unknown='ignore'))
])
num_pipe = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='most_frequent')),
])

transformer = ColumnTransformer([('categorical', cat_pipe, X_train_cat.columns),
                                 ('numerical', num_pipe, X_train_num.columns)])

In [8]:
model_pipe = Pipeline(steps=[
    ('trans', transformer),
    ('rfc', RandomForestClassifier(verbose=1, n_jobs=-2))
])

model_pipe.fit(X_train, y_train)

[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done  36 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-2)]: Done 100 out of 100 | elapsed:    0.5s finished


Pipeline(steps=[('trans',
                 ColumnTransformer(transformers=[('categorical',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  Index(['age_group', 'education', 'race', 'sex', 'income_poverty',
       'marital_status', 'rent_or_own', 'employment_status', 'hhs_geo_region',
       'census_msa'],
      dtype=...
       'behavioral_touch_face', 'doctor_recc_h1n1', 'doctor_recc_seasonal',
       'chronic_med_condition', 'child_under_6_months', 'health_worker',
       'opinion_h1n1_vacc_effective', 'opinion_h1n1_risk',
   

In [9]:
rfc_cv_score = cross_val_score(model_pipe, X_train, y_train, n_jobs=-1, verbose=3, scoring = 'f1')

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    4.8s remaining:    7.3s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    5.1s finished


In [10]:
rfc_cv_score.mean()

0.7500676601384576

# Random Forest Classifier

In [11]:
model_pipe_2 = Pipeline(steps=[
    ('trans', transformer),
    ('rfc', RandomForestClassifier(verbose=1, n_jobs=-2))
])

In [12]:
params = {
    'rfc__max_depth': list(range(10,100,10)),
    'rfc__criterion': ['gini', 'entropy'],
    'rfc__n_estimators': list(range(100,250,50)),
    'rfc__min_samples_leaf': list(range(2,10,2)),
    'rfc__min_samples_split': list(range(2,10,2)),
}

In [13]:
gs_rfc = GridSearchCV(model_pipe_2, params, n_jobs=-1, verbose=3, cv = 3, scoring = 'f1')
gs_rfc.fit(X_train, y_train)

Fitting 3 folds for each of 864 candidates, totalling 2592 fits


[Parallel(n_jobs=-2)]: Using backend ThreadingBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done  36 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-2)]: Done 100 out of 100 | elapsed:    0.6s finished


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('trans',
                                        ColumnTransformer(transformers=[('categorical',
                                                                         Pipeline(steps=[('impute',
                                                                                          SimpleImputer(strategy='most_frequent')),
                                                                                         ('ohe',
                                                                                          OneHotEncoder(handle_unknown='ignore',
                                                                                                        sparse=False))]),
                                                                         Index(['age_group', 'education', 'race', 'sex', 'income_poverty',
       'marital_status', 'rent_or_own', 'employment_status', 'hhs_geo_...
       'opinion_seas_risk', 'opinion_seas_sick_from

In [14]:
gs_rfc.best_params_

{'rfc__criterion': 'gini',
 'rfc__max_depth': 80,
 'rfc__min_samples_leaf': 2,
 'rfc__min_samples_split': 8,
 'rfc__n_estimators': 100}

In [16]:
rfc_cv_1 = cross_val_score(gs_rfc.best_estimator_, X_train, y_train, n_jobs=-1, verbose=3, scoring = 'f1')

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    1.5s remaining:    2.3s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    1.9s finished


In [17]:
rfc_cv_1.mean()

0.7564179759885572

## Gradiant Boost Classifier

In [18]:
model_pipe_3 = Pipeline(steps=[
    ('trans', transformer),
    ('gbc', GradientBoostingClassifier(verbose=3))
])

In [19]:
model_pipe_3.get_params()

{'memory': None,
 'steps': [('trans',
   ColumnTransformer(transformers=[('categorical',
                                    Pipeline(steps=[('impute',
                                                     SimpleImputer(strategy='most_frequent')),
                                                    ('ohe',
                                                     OneHotEncoder(handle_unknown='ignore',
                                                                   sparse=False))]),
                                    Index(['age_group', 'education', 'race', 'sex', 'income_poverty',
          'marital_status', 'rent_or_own', 'employment_status', 'hhs_geo_region',
          'census_msa'],
         dtype='object')),
                                   ('numerical',
                                    P...
          'behavioral_large_gatherings', 'behavioral_outside_home',
          'behavioral_touch_face', 'doctor_recc_h1n1', 'doctor_recc_seasonal',
          'chronic_med_condition', 'child_u

In [20]:
params = {
    'gbc__learning_rate': [0.001, 0.01, 0.1, .5, .9],
    'gbc__n_estimators': list(range(100,250,50)),
    'gbc__min_samples_leaf': list(range(2,10,2)),
    'gbc__min_samples_split': list(range(2,10,2)),
    'gbc__max_features': list(range(0,200,50))
}

In [21]:
gs_gbc = GridSearchCV(model_pipe_3, params, n_jobs=-1, verbose=1, cv = 3, scoring = 'f1')
gs_gbc.fit(X_train, y_train)

Fitting 3 folds for each of 960 candidates, totalling 2880 fits


C:\Users\Manav Kahlon\anaconda3\envs\learn-env\lib\site-packages\sklearn\model_selection\_search.py:922: UserWarning: One or more of the test scores are non-finite: [       nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
 0.36044927 0.55713292 0.65718802 0.36525133 0.52621018 0.65771569
 0.35187558 0.54556853 0.65888706 0.36044927 0.53267225 0.65727244
 0.36525133 0.54783608 0.65973234 0.36514228 0.51120114 0.65928445
 0.36044927 0.54389395 0.65718802 0.36514228 0.52493171 0.65950082
 0.36044927 0.54851143 0.659284

      Iter       Train Loss   Remaining Time 
         1           1.3295           11.51s
         2           1.2859           11.06s
         3           1.2505           12.12s
         4           1.2188           12.22s
         5           1.1924           11.79s
         6           1.1702           11.83s
         7           1.1501           11.91s
         8           1.1329           11.80s
         9           1.1174           11.51s
        10           1.1035           11.45s
        11           1.0921           11.26s
        12           1.0811           11.20s
        13           1.0707           11.06s
        14           1.0624           11.01s
        15           1.0537           10.81s
        16           1.0460           10.61s
        17           1.0387           10.47s
        18           1.0326           10.33s
        19           1.0272           10.16s
        20           1.0216            9.99s
        21           1.0162            9.84s
        2

       187           0.9016            0.52s
       188           0.9014            0.48s
       189           0.9011            0.44s
       190           0.9010            0.40s
       191           0.9008            0.36s
       192           0.9007            0.32s
       193           0.9006            0.28s
       194           0.9003            0.24s
       195           0.9002            0.20s
       196           0.9000            0.16s
       197           0.8998            0.12s
       198           0.8997            0.08s
       199           0.8996            0.04s
       200           0.8994            0.00s


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('trans',
                                        ColumnTransformer(transformers=[('categorical',
                                                                         Pipeline(steps=[('impute',
                                                                                          SimpleImputer(strategy='most_frequent')),
                                                                                         ('ohe',
                                                                                          OneHotEncoder(handle_unknown='ignore',
                                                                                                        sparse=False))]),
                                                                         Index(['age_group', 'education', 'race', 'sex', 'income_poverty',
       'marital_status', 'rent_or_own', 'employment_status', 'hhs_geo_...
       'opinion_seas_risk', 'opinion_seas_sick_from

In [22]:
gs_gbc.best_params_

{'gbc__learning_rate': 0.1,
 'gbc__max_features': 50,
 'gbc__min_samples_leaf': 2,
 'gbc__min_samples_split': 6,
 'gbc__n_estimators': 200}

In [23]:
gbc_cv_1 = cross_val_score(gs_gbc.best_estimator_, X_train, y_train, n_jobs=-1, verbose=3, scoring = 'f1')

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:   13.1s remaining:   19.8s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   13.3s finished


In [24]:
gbc_cv_1.mean()

0.7650133030437904